In [1]:
from bs4 import BeautifulSoup
import requests
from corus import load_buriy_webhose
from nltk.tokenize import sent_tokenize
import spacy
import re
import wget
from tqdm import tqdm
import json
import numpy as np
import pandas as pd

# 1. Получение значений

In [2]:
def clean(txt): # функция очистки теста словарной статьи из wikitionary
    text = re.sub(r'◆.+', '', txt)
    text = re.sub(r'\[[≈≠▲▼].+', '', text)
    text = re.sub(r'^[а-яё]+\. ', '', text)
    text = re.sub(r'\s', ' ', text)
    text = re.sub(r' (?=[\W])', '', text)
    return re.sub(r' \Z', '', text)

In [3]:
def getmeanings(word): # функция автоматического получения значений из wikitionary для русского
    bs = BeautifulSoup(requests.get(f'https://ru.wiktionary.org/wiki/{word.lower()}').text)
    meanings = [clean(el.text) for el in bs.find('ol').find_all('li')]
    return meanings

In [4]:
words = ['куча', 'золото', 'волна', 'выход', 'лист'] # слова, с которыми я буду работать

In [5]:
lexemes = {} # получение значений
for word in words:
    lexemes[word] = getmeanings(word)

Посмотрим на получившиеся значения:

In [6]:
lexemes

{'куча': ['большое количество чего-либо, обычно сыпучего, мелкого, наваленного, насыпанного в одном месте',
  'беспорядочное скопление людей, животных, автомобилей и т. п.',
  'большое количество чего-либо',
  'структура данных в виде дерева или деревьев, где у каждого узла значение ключа не меньше(в другом варианте: не больше), чем значения ключей у узлов-потомков',
  'область динамически распределяемой памяти'],
 'золото': ['химический элемент с атомным номером 79, обозначается химическим символом Au',
  'драгоценный металл жёлтого цвета',
  'то, что цветом напоминает такой металл; золотой цвет, блеск чего-либо',
  'нечто ценное',
  'обращение к тому, кто дорог и любим',
  'то же, что золотая медаль',
  'денежный эквивалент'],
 'волна': ['вал на поверхности водоёма при её колебании под действием ветра, сейсмических явлений, механического воздействия',
  'изменение некоторой совокупности физических величин(характеристик некоторого физического поля или материальной среды), способное пе

# 2. Отбор предложений из корпуса

## 2.1. Обработка корпуса

```python
corpus = load_buriy_webhose('webhose-2016.tar.bz2') # подгрузка корпуса webhose

nlp = spacy.load('ru_core_news_sm') # подгрузка модуля spacy

examples = {word: [] for word in words} # словарь для примеров

for article in tqdm(corpus, total=285965): # парсим статьи в корпусе
    for sentence in sent_tokenize(article.text): # делим статьи на предложения
        lemmas = [word.lemma_ for word in nlp(sentence)] # лемматизируем
        i = 0
        while i < len(words): # проходимся по таргетным словам
            # добавляем предложение к примерам, если в нем присутствует таргетное слово
            if words[i] in lemmas and sentence not in examples[words[i]]:  
                examples[words[i]].append(sentence)
                # если для слова уже набралось 200 примеров - больше не ищем для него примеры
                if len(examples[words[i]]) == 200: 
                    words.pop(i)
            i += 1
    if len(words) == 0:
        break

words = ['куча', 'золото', 'волна', 'выход', 'лист']
# записываем результат
with open('examples.json', 'w', encoding='utf-8') as f:
    json.dump(examples, f, ensure_ascii=False, indent='\t')
```

## 2.2. Загрузка данных

In [7]:
with open('examples.json', 'r', encoding='utf-8') as f: # читаем данные
    examples = json.load(f)

Посмотрим на примеры истатистику:

In [8]:
for word, example_list in examples.items():
    print(f'{word}, примеров {len(example_list)}', '---', *example_list[:3], sep='\n')
    print('\n')

куча, примеров 101
---
Но тогда хозяева не использовали кучу моментов, сравняв счет лишь в концовке.
Слушаешь, и начинаешь понимать – острее всех люди реагируют на, казалось бы, мелочи... 
Ну вот Дональд Трамп наговорил кучу страшнейших вещей - от изгнания мусульман до признания Крыма частью России.
В ответ критики заявили, что тоже могут поехать США, посетить NASA в качестве туристов и наснимать кучу фотографий.


золото, примеров 165
---
Фото: Архив пресс-службы Стивен Вебстер посвятил коллекцию гаданию на цветке В коллекцию Stephen Webster «Love Me, Love Me Not» вошли украшения из розового и белого золота, инкрустированные зеленым агатом в окружении черных бриллиантов и розовым опалом или гематитом с белыми бриллиантами.
По словам министра, которые приводит агентство Рейтер, соглашение об ограничении добычи "черного золота" будет действовать до одного года.
Goldman Sachs вычислил, когда завершится «нефтяное ралли» 6 Октября 2016, 20:18 
Ралли на мировом нефтяном рынке закончится, ко

# 3. Классификация

In [9]:
import torch
from transformers import BertModel, BertTokenizerFast
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize 
import nltk

## 3.1. Контекстные эмбеддинги

Подгружаю небольшую [модель](https://huggingface.co/setu4993/smaller-LaBSE) для создания эмбеддингов для предложений.

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("setu4993/smaller-LaBSE")
model = BertModel.from_pretrained("setu4993/smaller-LaBSE")

In [11]:
embeddings = {word: [] for word in words}

for word, examples_list in examples.items(): # беру примеры для каждого слова
    embeddings[word] = [] # создаю список эмбеддингов
    for i in range(0, len(examples_list), 20): # делю примеры на батчи по 20 вхождений
        tokenized = tokenizer(examples_list[i:i+20], return_tensors='pt', padding=True) # токенизирую примеры
        embeddings[word].extend(model(**tokenized).pooler_output.tolist()) # получаю эмбеддинги

In [12]:
%%capture

clustering = {}
for word, emb in embeddings.items():
    # кластеризую примеры методом KMeans, где k вдвое больше количества значений слова;
    # евристика следующая: чем больше значений в словаре, тем больше возможных контекстов
    clustering[word] = KMeans(len(lexemes[word])*2, random_state=0).fit(emb) 

## 3.2. TF-IDF

Пробую кластеризацию по матрице TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
nlp = spacy.load('ru_core_news_sm') # подгрузка модуля spacy
matrices = {word: {} for word in words}

for word, examples_list in examples.items(): # беру примеры для каждого слова
    # лемматизирую их
    lemmatized = [' '.join([word.lemma_ for word in nlp(sentence)]) for sentence in examples_list]
    
    tfidf = TfidfVectorizer(tokenizer=lambda x: x.split()) # создаю токенизатор
    matrices[word]['matrix'] = tfidf.fit_transform(lemmatized) # получаю матрицу для примеров
    matrices[word]['vectorizer'] = tfidf # сохраняю токенизатор

D:\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [17]:
%%capture

clustering_tfidf = {}
for word, val in matrices.items():
    # кластеризую примеры методом KMeans, где k вдвое больше количества значений слова;
    # евристика следующая: чем больше значений в словаре, тем больше возможных контекстов
    clustering_tfidf[word] = KMeans(len(lexemes[word])*2, random_state=0).fit(val['matrix'])

# 4. Классификация словарных значений

## 4.1. Контекстные эмбеддинги

In [18]:
classes = {}

for word, meanings in lexemes.items(): # беру значения каждого слова
    tokenized = tokenizer(meanings, return_tensors='pt', padding=True) # токенизирую их
    emb = model(**tokenized).pooler_output.tolist() # получаю эмбеддинги
    classes[word] = clustering[word].predict(emb) # кластеризую тем же способом, что и примеры для данного слова

Посмотрим на получившуюся классификацию:

In [19]:
classes

{'куча': array([1, 6, 6, 1, 0]),
 'золото': array([ 6,  8,  5,  5, 12, 12,  6]),
 'волна': array([1, 9, 6, 5, 2]),
 'выход': array([ 7,  7,  7,  7, 10, 10,  0]),
 'лист': array([3, 3, 4])}

## 4.2. TF-IDF

In [20]:
classes_tfidf = {}

for word, meanings in lexemes.items(): # беру значения каждого слова
    lemmatized = [' '.join([word.lemma_ for word in nlp(sentence)]) for sentence in meanings] # лемматизирую их
    # получаю вектора тем же способом, что и для примеров с этим словом
    vectors = matrices[word]['vectorizer'].transform(lemmatized)     
    classes_tfidf[word] = clustering_tfidf[word].predict(vectors) # кластеризую тем же способом, что и примеры для данного слова

Посмотрим на получившуюся классификацию:

In [21]:
classes_tfidf

{'куча': array([8, 0, 8, 8, 0]),
 'золото': array([0, 3, 3, 3, 3, 3, 3]),
 'волна': array([5, 9, 5, 6, 9]),
 'выход': array([11,  7, 11, 11, 11,  4, 11]),
 'лист': array([2, 3, 3])}

# 5. Разметка

Я решил взять те значения, которые в обеих классификациях попали в разные классы

In [22]:
chosen = {'куча': [0, 1],
 'золото': [0, 1],
 'волна': [0, 2],
 'выход': [3, 5, 6],
 'лист': [0, 1]}

## 5.1. Контекстные эмбеддинги

In [23]:
out = {}
for word, meanings in chosen.items(): # прохожусь по словам
    out[word] = []
    for meaning in meanings: # прохожусь по значениям
        # получаю тескт для значения и номер класса, к которому относится значения
        text, num = lexemes[word][meaning], classes[word][meaning] 
        examples_num = np.where(clustering[word].labels_ == num)[0][:5] # получаю индексы примеров из того же класса
        examples_text = np.array(examples[word])[examples_num] # получаю примеры по индексам
        out[word].append({'meaning': text, 'examples': examples_text.tolist()}) # добавляю в словарь

In [24]:
df = pd.DataFrame(columns=['word', 'meaning', 'example', 'marker']) # формирую таблицу для разметки
for word, meanings in out.items(): # прохожусь по словам
    for meaning in meanings: # прохожусь по значениям
        new = pd.DataFrame(meaning['examples'], columns=['example']) # создаю столбец с примерами
        new['meaning'] = meaning['meaning'] # добавляю столбец со значением
        new['word'] = word # добавляю столбец со словом
        df = pd.concat((df, new)) # объединяю с общей таблицей

In [25]:
with pd.ExcelWriter('tagging_1.xlsx') as wt: # записываю в файл
    df.to_excel(wt, sheet_name='tagging', index=False)

## 5.2. TF-IDF

Здесь все аналогично

In [39]:
out_tfidf = {}
for word, meanings in chosen.items():
    out_tfidf[word] = []
    for meaning in meanings:
        text, num = lexemes[word][meaning], classes_tfidf[word][meaning]
        examples_num = np.where(clustering_tfidf[word].labels_ == num)[0][:5]
        examples_text = np.array(examples[word])[examples_num]
        out_tfidf[word].append({'meaning': text, 'examples': examples_text.tolist()})

In [40]:
df_tfidf = pd.DataFrame(columns=['word', 'meaning', 'example', 'marker'])
for word, meanings in out_tfidf.items():
    for meaning in meanings:
        new = pd.DataFrame(meaning['examples'], columns=['example'])
        new['meaning'] = meaning['meaning']
        new['word'] = word
        df_tfidf = pd.concat((df_tfidf, new))

In [41]:
with pd.ExcelWriter('tagging_2.xlsx') as wt:
    df_tfidf.to_excel(wt, sheet_name='tagging', index=False)

# 6. Оценка качества

После ручной разметки можео подвести итоги

In [42]:
accuracy_dl, accuracy_tfidf = pd.read_excel('tagging_1.xlsx').marker.mean(), pd.read_excel('tagging_2.xlsx').marker.mean()

In [43]:
print(f'Accuracy на основе работы BERT модели составляет {accuracy_dl}\n'
     f'Accuracy на основе матрицы TF-IDF составляет {accuracy_tfidf}\n')

Accuracy на основе работы BERT модели составляет 0.21818181818181817
Accuracy на основе матрицы TF-IDF составляет 0.21818181818181817



# 7. Сравнение результатов

В итоге значения accuracy получились одинаковыми, при этом распределение ошибок немного отличается:

In [2]:
pd.read_excel('tagging_1.xlsx').groupby(['word', 'meaning']).aggregate({'marker': np.mean})

marker
word   meaning                                                   
волна  вал на поверхности водоёма при её колебании под...     0.8
       густой поток, массовый наплыв чего-либо                0.6
выход  количество произведённых продуктов(например, на...     0.0
       попадание на определённый этап конкурса или сор...     0.0
       силовой элемент на турнике, конечная цель котор...     0.0
золото драгоценный металл жёлтого цвета                       0.6
       химический элемент с атомным номером 79, обозна...     0.0
куча   беспорядочное скопление людей, животных, автомо...     0.2
       большое количество чего-либо, обычно сыпучего, ...     0.2
лист   орган растения в виде тонкой пластинки, служащи...     0.0
       пласт, тонкий плоский слой                             0.0

In [3]:
pd.read_excel('tagging_2.xlsx').groupby(['word', 'meaning']).aggregate({'marker': np.mean})

marker
word   meaning                                                   
волна  вал на поверхности водоёма при её колебании под...     0.0
       густой поток, массовый наплыв чего-либо                0.8
выход  количество произведённых продуктов(например, на...     0.0
       попадание на определённый этап конкурса или сор...     0.2
       силовой элемент на турнике, конечная цель котор...     0.0
золото драгоценный металл жёлтого цвета                       1.0
       химический элемент с атомным номером 79, обозна...     0.0
куча   беспорядочное скопление людей, животных, автомо...     0.0
       большое количество чего-либо, обычно сыпучего, ...     0.4
лист   орган растения в виде тонкой пластинки, служащи...     0.0
       пласт, тонкий плоский слой                             0.0

Оба подхода не справились со значениями, которые являются довольно специфичными для той или иной области: **выход** (_силовой элемент на турнике_), **вызод** (_количество произведённых продуктов_), **золото** (_химический элемент_), а также со значениями, которые скорее были плохо представлены в примерах из-за специфики самого корпуса, как например приведённые здесь значения слова **лист** (а в корпусе в основном встречается значение _лист бумаги_).

Качество подходов незначительно отличается друг от друга из-за того, что в конечном счёте у них похожий принцип: мы пытаемся вывести значение слова через его контекст, отличается лишь методология работы с контекстом. На мой взгляд, для более успешного решения задачи нужно больше сконцентрироваться самом слове, так как контекст целиком может быть чувствителен к другим вхождениям, даже если они напрямую не касаются нашего таргетного слова. Таким образом, в подходе с эмбеддингами нейросети более успешным решением было бы анализировать эмбеддинг таргетного слова в контексте.